# Problem Description

이 튜토리얼에서 시퀀스 학습을 시연하는 데 사용할 문제는 IMDB 영화 검토 감정 분류 문제이다. [IMDB movie review](http://ai.stanford.edu/~amaas/data/sentiment/) 

- 이 데이터는 스탠포드 연구진에 의해 수집되었으며 데이터의 50-50 분할이 훈련과 테스트에 사용된 2011년 논문에서 사용되었다. 88.89%의 정확도가 달성되었다.
- Keras는 IMDB 내장 데이터셋에 대한 액세스를 제공합니다. imdb.load_data() 함수를 사용하여 데이터 집합을 로드할 수 있습니다.
- 각 영화 리뷰는 단어 순서가 다양하며 각 영화 리뷰의 감정은 분류되어야 한다.
- 우리는 각 영화 리뷰를 단어 임베딩이라는 텍스트로 작업할 때 널리 사용되는 기술인 실제 벡터 영역으로 매핑할 것이다.

    - 우리는 각 단어를 32길이의 실제 가치 벡터에 매핑할 것이다.
    - 또한 모델링에 관심이 있는 총 단어 수를 5000개의 가장 자주 사용하는 단어로 제한하고 나머지는 0개로 제한할 예정입니다.

In [1]:
import time

start = time.time()
print('시작시간:', time.ctime(start))

시작시간: Fri Sep 24 13:53:03 2021


In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

# Simple LSTM for Sequence Classification

## Step 1: Data Preparation

In [3]:
# load the dataset but only keep the top n words, zero the rest

top_words = 5000 # 단어 수집 개수
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
# truncate and pad input sequences

max_review_length = 500 # 단어 범위
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

## Step 2 ~ 4: Build, compile and fit the model.

In [5]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))  # 100 memory units (smart neurons)
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64) 
# The model is fit for only 3 epochs because it quickly overfits the problem. 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 47s 102ms/step - loss: 0.5388 - accuracy: 0.7263 - val_loss: 0.3758 - val_accuracy: 0.8416
Epoch 2/3
391/391 [==============================] - 39s 100ms/step - loss: 0.3519 - accuracy: 0.8528 - val_loss: 0.3624 - val_accuracy: 0.8478
Epoch 3/3
391/391 [==============================] - 39s 100ms/step - loss: 0.3223 - ac

## Step 5: Evaluate the model.

In [6]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.62%


이 간단한 LSTM은 거의 조정되지 않고 IMDB 문제에 대해 거의 최신 결과를 얻을 수 있습니다. [state-of-the-art results](https://paperswithcode.com/sota/sentiment-analysis-on-imdb)

# LSTM For Sequence Classification With Dropout

In [7]:
# LSTM with Dropout for sequence classification in the IMDB dataset
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))   # newly added
model.add(LSTM(100))
model.add(Dropout(0.2))   # newly added
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64,validation_split=0.2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
313/313 [==============================] - 29s 85ms/step - loss: 0.5008 - accuracy: 0.7452 - val_lo

우리는 융합이 약간 느린 추세와 이 경우 최종 정확도가 낮은 중퇴자가 교육에 원하는 영향을 미치는 것을 볼 수 있다. <br>이 모델은 몇 가지 더 많은 교육 기간을 사용할 수 있으며 더 높은 기술을 얻을 수 있습니다.

# LSTM For Sequence Classification With [Dropout and Recurrent Dropout](https://stackoverflow.com/questions/49940280/keras-lstm-dropout-vs-recurrent-dropout)

- LSTM을 사용하여 메모리 유닛의 입력 및 반복 연결에 드롭아웃을 정밀하고 별도로 적용할 수 있다.

- Keras는 LSTM 계층의 매개 변수, 입력 드롭아웃을 구성하기 위한 드롭아웃 및 반복 드롭아웃을 통해 이 기능을 제공합니다.

- 반복 드롭아웃은 반복 신경망을 위한 정규화 방법이다. 드롭아웃은 LSTM 메모리 셀(또는 GRU 상태) 업데이트에 적용된다. 즉, LSTM/GRU의 입력/업데이트 게이트를 드롭한다.

> 드롭아웃: [0,1], 입력의 선형 변환을 위해 삭제할 단위의 비율입니다.
> 
> recurrent_dropout: [0,1], recurrent 상태의 선형 변환을 위해 삭제할 단위의 비율입니다.
>

In [8]:
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # newly modified
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 1105s 3s/step - loss: 0.4584 - accuracy: 0.7707
Epoch 2/3
391/391 [==============================] - 1113s 3s/step - loss: 0.3063 - accuracy: 0.8769
Epoch 3/3
391/391 [==============================] - 1129s 3s/step - loss: 0.2751 - accuracy: 0.8902
Accuracy: 87.14%


- LSTM 특정 드롭아웃이 계층별 드롭아웃보다 네트워크 컨버전스에 더 뚜렷한 영향을 미친다는 것을 알 수 있다.

- 위와 같이, 에포크의 수는 일정하게 유지되었고 모델의 기술을 더 높일 수 있는지 확인하기 위해 증가할 수 있었다.

# LSTM and Convolutional Neural Network For Sequence Classification

- 컨볼루션 신경망은 입력 데이터의 공간 구조를 학습하는 데 탁월하다.
- IMDB 리뷰 데이터는 리뷰의 단어 순서로 1차원 공간 구조를 가지고 있으며 CNN은 긍정과 부정에 대해 불변 특징을 선택할 수 있다.
- 이렇게 학습된 공간 특성은 LSTM 계층에 의해 시퀀스로 학습될 수 있다.
- 임베딩 레이어 다음에 1차원 CNN 및 최대 풀링 레이어를 쉽게 추가할 수 있으며, 이 레이어는 통합된 기능을 LSTM에 제공한다.
    - 필터 길이가 3인 작은 32개 기능 세트를 사용할 수 있습니다.
    - 풀링 계층은 표준 길이 2를 사용하여 형상도 크기를 절반으로 줄일 수 있습니다.

In [9]:
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))   # newly added
model.add(MaxPooling1D(pool_size=2))    # newly added
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 46s 43ms/step - loss: 0.5697 - accuracy: 0.7071
Epoch 2/

교육 시간은 빨라지지만 첫 번째 예제와 유사한 결과를 얻을 수 있습니다.

이 예제를 중퇴자를 사용하기 위해 더 확장한다면 훨씬 더 나은 결과를 얻을 수 있을 것으로 기대합니다.

In [10]:
execution_time = time.time() - start
print('실행시간(분): ', execution_time/60)

실행시간(분):  65.12580471038818
